In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv("../data/interim/df_with_ner.csv", index_col=0)

In [3]:
df

,Name,Race,luke,flair_results,num_luke_entities,true_num_entities,all_person_entities
0,Scott,White,Scott PER\n,"Span [1]: ""Scott"" [− Labels: PER (0.9999)]\n",1,1,True
1,Kathleen,White,Kathleen PER\n,"Span [1]: ""Kathleen"" [− Labels: PER (0.8167)]\n",1,1,True
2,Matthew,White,NaN,"Span [1]: ""Matthew"" [− Labels: PER (0.4564)]\n",0,1,True
3,Jeffrey,White,Jeffrey PER\n,"Span [1]: ""Jeffrey"" [− Labels: PER (0.9686)]\n",1,1,True
4,Mark,White,NaN,"Span [1]: ""Mark"" [− Labels: LOC (0.5577)]\n",0,1,True
...,...,...,...,...,...,...,...
25423,mario tejeda went to the store.,Hispanic,NaN,"Span [1,2]: ""mario tejeda"" [− Labels: PER (1...",0,6,True
25424,mario tavarez went to the store.,Hispanic,NaN,"Span [1,2]: ""mario tavarez"" [− Labels: PER (...",0,6,True
25425,mario taveras went to the store.,Hispanic,NaN,"Span [1,2]: ""mario taveras"" [− Labels: PER (...",0,6,True
25426,mario villatoro went to the store.,Hispanic,NaN,"Span [1,2]: ""mario villatoro"" [− Labels: PER...",0,6,True


In [4]:
flair_df = df[["Name", "Race", "flair_results", "true_num_entities"]]

In [5]:
# TODO: make sure that this function performs well in all edge cases
def get_num_flair_entities(input_string):
    if str(input_string) != "nan":
        total_entities = 0
        input_string = str(input_string)
        query_two_string = ']: "'
        query_three_string = '"   ['    
        start_index = input_string.index(query_two_string) + len(query_two_string)
        end_index = input_string.index(query_three_string)
        value = input_string[start_index:end_index]

        total_entities += (1 + value.count(" "))
        return total_entities
    return 0


In [6]:
# query_two_string = ']: "'
# query_three_string = '"   ['    
# start_index = input_string.index(query_two_string) + len(query_two_string)


In [7]:
flair_df["flair_num_entities"] = flair_df["flair_results"].apply(lambda x: get_num_flair_entities(x))

/var/folders/8_/p6s91lnx1vn9_y920h68jhy00000gn/T/ipykernel_24321/1888831580.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flair_df["flair_num_entities"] = flair_df["flair_results"].apply(lambda x: get_num_flair_entities(x))


In [8]:
flair_df

,Name,Race,flair_results,true_num_entities,flair_num_entities
0,Scott,White,"Span [1]: ""Scott"" [− Labels: PER (0.9999)]\n",1,1
1,Kathleen,White,"Span [1]: ""Kathleen"" [− Labels: PER (0.8167)]\n",1,1
2,Matthew,White,"Span [1]: ""Matthew"" [− Labels: PER (0.4564)]\n",1,1
3,Jeffrey,White,"Span [1]: ""Jeffrey"" [− Labels: PER (0.9686)]\n",1,1
4,Mark,White,"Span [1]: ""Mark"" [− Labels: LOC (0.5577)]\n",1,1
...,...,...,...,...,...
25423,mario tejeda went to the store.,Hispanic,"Span [1,2]: ""mario tejeda"" [− Labels: PER (1...",6,2
25424,mario tavarez went to the store.,Hispanic,"Span [1,2]: ""mario tavarez"" [− Labels: PER (...",6,2
25425,mario taveras went to the store.,Hispanic,"Span [1,2]: ""mario taveras"" [− Labels: PER (...",6,2
25426,mario villatoro went to the store.,Hispanic,"Span [1,2]: ""mario villatoro"" [− Labels: PER...",6,2


In [9]:
ex_tough_output = 'Span [1,2]: "George Washington"   [− Labels: PER (0.9993)]\nSpan [8]: "Washington"   [− Labels: LOC (0.9929)]\nSpan [13]: "Indiana"   [− Labels: LOC (0.9993)]\n'

In [10]:
ex_string = 'Span [1,2]: "mario gudino"   [− Labels: PER (0.9954)]\n'

In [11]:
def check_all_person_entities(input_string):
    input_string = str(input_string)
    if input_string != "nan":
        for i in input_string.split("\n"):
            if len(i) > 0:
                query_string = '"   [− Labels: '
                start_index = i.index(query_string) + len(query_string)
                value = i[start_index:start_index+4]
                if value != "PER ":
                    return False
                
        return True
    return None

In [12]:
check_all_person_entities(ex_tough_output)

False

In [13]:
flair_df["all_person_entities"] = flair_df["flair_results"].apply(lambda x: check_all_person_entities(x))

/var/folders/8_/p6s91lnx1vn9_y920h68jhy00000gn/T/ipykernel_24321/1930742301.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flair_df["all_person_entities"] = flair_df["flair_results"].apply(lambda x: check_all_person_entities(x))


In [14]:
mislabeled = flair_df.loc[(flair_df["all_person_entities"] != True) | (flair_df["flair_num_entities"] != flair_df["true_num_entities"])]

In [26]:
mislabeled["Race"].value_counts()

API         3646
Black       3303
Hispanic    2401
White       2280
Name: Race, dtype: int64

In [16]:
get_num_flair_entities(ex_string)

2

In [17]:
flair_df.iloc[-1]["flair_results"]

'Span [1,2]: "mario gudino"   [− Labels: PER (1.0)]\n'

In [18]:
flair_df["len_res"] = flair_df["flair_results"].apply(lambda x: len(str(x)))

In [19]:
flair_df.sort_values(by="len_res", ascending=False)

,Name,Race,flair_results,true_num_entities,flair_num_entities,all_person_entities,len_res
21868,hung krishnamurthy went to the store.,API,"Span [1]: ""hung"" [− Labels: PER (0.9993)]\nS...",6,1,True,97
13392,hung krishnamurthy,API,"Span [1]: ""hung"" [− Labels: PER (0.9986)]\nS...",2,1,True,97
22668,sang krishnamurthy went to the store.,API,"Span [1]: ""sang"" [− Labels: PER (0.9999)]\nS...",6,1,True,96
19224,latanya jeanlouis went to the store.,Black,"Span [1]: ""latanya"" [− Labels: LOC (0.8862)]...",6,1,False,96
23168,dung krishnamurthy went to the store.,API,"Span [1]: ""dung"" [− Labels: PER (0.9999)]\nS...",6,1,True,96
...,...,...,...,...,...,...,...
22995,young oh went to the store.,API,NaN,6,0,None,3
22996,young liou went to the store.,API,NaN,6,0,None,3
22997,young ramakrishnan went to the store.,API,NaN,6,0,None,3
22998,young ramanathan went to the store.,API,NaN,6,0,None,3


# LUKE Analysis

In [20]:
luke_df = df[["Name", "Race", "luke", "num_luke_entities", "true_num_entities", "all_person_entities"]]

In [21]:
luke_df

,Name,Race,luke,num_luke_entities,true_num_entities,all_person_entities
0,Scott,White,Scott PER\n,1,1,True
1,Kathleen,White,Kathleen PER\n,1,1,True
2,Matthew,White,NaN,0,1,True
3,Jeffrey,White,Jeffrey PER\n,1,1,True
4,Mark,White,NaN,0,1,True
...,...,...,...,...,...,...
25423,mario tejeda went to the store.,Hispanic,NaN,0,6,True
25424,mario tavarez went to the store.,Hispanic,NaN,0,6,True
25425,mario taveras went to the store.,Hispanic,NaN,0,6,True
25426,mario villatoro went to the store.,Hispanic,NaN,0,6,True


In [22]:
def is_correct(x):    
    return (x["all_person_entities"]) and (x["num_luke_entities"] == x["true_num_entities"])

luke_df["correct_result"] = luke_df.apply(is_correct, axis=1) 


In [23]:
luke_df

,Name,Race,luke,num_luke_entities,true_num_entities,all_person_entities,correct_result
0,Scott,White,Scott PER\n,1,1,True,True
1,Kathleen,White,Kathleen PER\n,1,1,True,True
2,Matthew,White,NaN,0,1,True,False
3,Jeffrey,White,Jeffrey PER\n,1,1,True,True
4,Mark,White,NaN,0,1,True,False
...,...,...,...,...,...,...,...
25423,mario tejeda went to the store.,Hispanic,NaN,0,6,True,False
25424,mario tavarez went to the store.,Hispanic,NaN,0,6,True,False
25425,mario taveras went to the store.,Hispanic,NaN,0,6,True,False
25426,mario villatoro went to the store.,Hispanic,NaN,0,6,True,False


In [24]:
luke_df.loc[luke_df["correct_result"] == False]["Race"].value_counts()

API         4272
Black       4048
Hispanic    3689
White       2740
Name: Race, dtype: int64

In [25]:
for i, row in luke_df.loc[(luke_df["correct_result"] == False) & (luke_df["Race"]=="White")].iterrows():
    print(row["Name"])

Matthew
Mark
Stephen
Thomas
Donald
Robert
Yoder
Mast
Hummel
Tierney
Toth
Yost
Funk
scott
kathleen
matthew
jeffrey
mark
stephen
susan
timothy
brian
thomas
gary
steven
donald
karen
christopher
william
robert
jennifer
john
michael
troyer
yoder
schulte
wisniewski
mast
mcdonough
stauffer
hummel
kaminski
tierney
schmitz
nowak
epstein
krueger
maurer
albrecht
schreiber
coughlin
ouellette
mueller
fink
siegel
kauffman
kowalski
omalley
schrader
herrmann
koch
gustafson
schumacher
toth
beyer
maguire
horvath
silverman
weiner
pfeiffer
weinstein
moeller
scherer
schwartz
schmitt
novak
koehler
knutson
schneider
seitz
schroeder
haas
sheehan
greenberg
metzger
kraus
flaherty
zimmer
pelletier
schwab
hoffmann
sauer
schaefer
huber
krause
mcnamara
schafer
schulz
swenson
stahl
ludwig
yost
cronin
diehl
caudill
mcgovern
ulrich
schilling
schwarz
monahan
baumann
devries
houck
bollinger
klein
kramer
weiss
odonnell
friedman
goldberg
schmidt
funk
kruse
jacobsen
mchugh
eckert
gerber
dietrich
goetz
bouchard
noonan
scott

stephen krause went to the store.
stephen mcnamara went to the store.
stephen schafer went to the store.
stephen schulz went to the store.
stephen swenson went to the store.
stephen stahl went to the store.
stephen ludwig went to the store.
stephen yost went to the store.
stephen cronin went to the store.
stephen diehl went to the store.
stephen caudill went to the store.
stephen mcgovern went to the store.
stephen ulrich went to the store.
stephen schilling went to the store.
stephen schwarz went to the store.
stephen monahan went to the store.
stephen baumann went to the store.
stephen devries went to the store.
stephen steinberg went to the store.
stephen houck went to the store.
stephen bollinger went to the store.
stephen klein went to the store.
stephen kramer went to the store.
stephen weiss went to the store.
stephen odonnell went to the store.
stephen friedman went to the store.
stephen goldberg went to the store.
stephen schmidt went to the store.
stephen funk went to the sto

christopher kramer went to the store.
christopher weiss went to the store.
christopher odonnell went to the store.
christopher friedman went to the store.
christopher goldberg went to the store.
christopher schmidt went to the store.
christopher funk went to the store.
christopher kruse went to the store.
christopher jacobsen went to the store.
christopher mchugh went to the store.
christopher eckert went to the store.
christopher gerber went to the store.
christopher dietrich went to the store.
christopher goetz went to the store.
christopher bouchard went to the store.
christopher noonan went to the store.
william troyer went to the store.
william yoder went to the store.
william schulte went to the store.
william wisniewski went to the store.
william mast went to the store.
william mcdonough went to the store.
william stauffer went to the store.
william hummel went to the store.
william kaminski went to the store.
william tierney went to the store.
william schmitz went to the store.

# Flair Finetuning Scratch:

